In [ ]:
# 1. Environment Setup and Imports (Enhanced for Thesis)
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from typing import List, Dict, Tuple, Optional
import json
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Core ML libraries for thesis models
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForCausalLM,
    BlipProcessor, BlipForQuestionAnswering,  # For BLIP VQA
    T5Tokenizer, T5ForConditionalGeneration,  # For T5/FLAN-T5
    CLIPProcessor, CLIPModel, 
    pipeline  # For easy model loading
)
from sentence_transformers import SentenceTransformer
import faiss

# Translation for multimodal pipeline
from googletrans import Translator
translator = Translator()

# RAG and LangChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Fine-tuning libraries
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import accelerate

# Evaluation and comparison
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import sacrebleu
from sklearn.metrics import accuracy_score, f1_score as sk_f1_score
import time

# Utils
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

print("=== BongoRAG Thesis Implementation ===")
print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("Running on CPU - consider using GPU for better performance")


In [ ]:
# 2. Configuration and Constants (Enhanced for Thesis Comparison)
class BongoRAGConfig:
    """Configuration class for the BongoRAG system with multiple model support"""
    
    # Dataset paths
    DATASET_PATH = "80k-bangla-qa-dataset.csv"
    VECTOR_STORE_PATH = "faiss_index"
    MODEL_CACHE_DIR = "./models"
    RESULTS_DIR = "./results"
    
    # Model configurations for comparison
    BANGLA_EMBEDDING_MODEL = "shihab17/bangla-sentence-transformer"
    
    # Multimodal models for thesis
    BLIP_VQA_MODEL = "Salesforce/blip-vqa-base"  # Primary VQA model
    BLIP_CAPTION_MODEL = "Salesforce/blip-image-captioning-base"  # For captions
    MINICPM_MODEL = "openbmb/MiniCPM-o-2_6"  # Alternative vision-language model
    
    # Text generation models for comparison
    T5_MODEL = "google/flan-t5-base"  # Accessible and good for QA
    T5_LARGE_MODEL = "google/flan-t5-large"  # For comparison
    BANGLA_GPT_MODEL = "csebuetnlp/banglat5_banglaparaphrase"  # Bangla-specific
    
    # Baseline models
    CLIP_MODEL = "openai/clip-vit-base-patch32"
    
    # RAG parameters
    CHUNK_SIZE = 512
    CHUNK_OVERLAP = 50
    TOP_K_RETRIEVAL = 5
    EMBEDDING_DIM = 768
    
    # Generation parameters
    MAX_LENGTH = 512
    MAX_NEW_TOKENS = 128
    TEMPERATURE = 0.7
    TOP_P = 0.9
    NUM_BEAMS = 4
    
    # LoRA configuration
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.1
    
    # Training parameters
    BATCH_SIZE = 4
    LEARNING_RATE = 2e-4
    NUM_EPOCHS = 3
    
    # Evaluation parameters
    EVAL_BATCH_SIZE = 8
    TEST_SIZE = 0.2
    
    # Thesis-specific parameters
    COMPARISON_MODELS = [
        "blip_vqa",
        "flan_t5_base", 
        "flan_t5_large",
        "bangla_t5",
        "minicpm_o"
    ]
    
    # Translation settings
    ENABLE_TRANSLATION = True
    TRANSLATE_CONFIDENCE_THRESHOLD = 0.5
    
    def __init__(self):
        os.makedirs(self.MODEL_CACHE_DIR, exist_ok=True)
        os.makedirs(self.RESULTS_DIR, exist_ok=True)

config = BongoRAGConfig()
print("Enhanced configuration loaded for thesis comparison!")
print(f"Models to compare: {config.COMPARISON_MODELS}")
print(f"Results will be saved to: {config.RESULTS_DIR}")


In [ ]:
# 3. Data Loading and Preprocessing
class BanglaDataProcessor:
    """Handles Bangla text preprocessing and data loading"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        
    def load_dataset(self) -> pd.DataFrame:
        """Load and preprocess the Bangla QA dataset"""
        print("Loading Bangla QA dataset...")
        df = pd.read_csv(self.config.DATASET_PATH)
        
        # Basic data cleaning
        df = df.dropna()
        df['Question'] = df['Question'].astype(str)
        df['Answer'] = df['Answer'].astype(str)
        
        # Remove any empty strings
        df = df[(df['Question'].str.strip() != '') & (df['Answer'].str.strip() != '')]
        
        print(f"Dataset loaded: {len(df)} QA pairs")
        return df
    
    def preprocess_text(self, text: str) -> str:
        """Clean and normalize Bangla text"""
        if not isinstance(text, str):
            return ""
        
        # Basic cleaning - you can extend this based on your needs
        text = text.strip()
        # Remove extra whitespaces
        text = ' '.join(text.split())
        
        return text
    
    def create_context_documents(self, df: pd.DataFrame) -> List[Document]:
        """Create LangChain documents from the dataset"""
        documents = []
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating documents"):
            # For RAG, we use answers as context and questions as metadata
            question = self.preprocess_text(row['Question'])
            answer = self.preprocess_text(row['Answer'])
            
            # Create a document with the answer as content
            doc = Document(
                page_content=answer,
                metadata={
                    'question': question,
                    'answer': answer,
                    'doc_id': idx
                }
            )
            documents.append(doc)
            
            # Also create a document with question-answer pair as content
            combined_content = f"প্রশ্ন: {question}\nউত্তর: {answer}"
            doc_combined = Document(
                page_content=combined_content,
                metadata={
                    'question': question,
                    'answer': answer,
                    'doc_id': f"{idx}_combined",
                    'type': 'qa_pair'
                }
            )
            documents.append(doc_combined)
        
        print(f"Created {len(documents)} documents for RAG")
        return documents

# Initialize data processor and load data
data_processor = BanglaDataProcessor(config)
df = data_processor.load_dataset()

# Display basic statistics
print("\n=== Dataset Statistics ===")
print(f"Total QA pairs: {len(df)}")
print(f"Average question length: {df['Question'].str.len().mean():.1f} characters")
print(f"Average answer length: {df['Answer'].str.len().mean():.1f} characters")
print("\nSample QA pairs:")
for i in range(3):
    print(f"Q: {df.iloc[i]['Question']}")
    print(f"A: {df.iloc[i]['Answer']}")
    print("---")


In [ ]:
# 4. Embedding Models and Vector Store Setup
class BanglaEmbeddingManager:
    """Manages embedding models for Bangla text and multimodal content"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        self.text_encoder = None
        self.clip_model = None
        self.clip_processor = None
        
    def load_bangla_embeddings(self):
        """Load Bangla sentence transformer"""
        print("Loading Bangla sentence transformer...")
        self.text_encoder = SentenceTransformer(
            self.config.BANGLA_EMBEDDING_MODEL,
            cache_folder=self.config.MODEL_CACHE_DIR
        )
        print("Bangla embeddings loaded successfully!")
        
    def load_clip_model(self):
        """Load CLIP model for multimodal processing"""
        print("Loading CLIP model...")
        self.clip_model = CLIPModel.from_pretrained(
            self.config.CLIP_MODEL,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        self.clip_processor = CLIPProcessor.from_pretrained(
            self.config.CLIP_MODEL,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        print("CLIP model loaded successfully!")
    
    def encode_texts(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Encode texts using Bangla sentence transformer"""
        if self.text_encoder is None:
            self.load_bangla_embeddings()
        
        print(f"Encoding {len(texts)} texts...")
        embeddings = self.text_encoder.encode(
            texts, 
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        return embeddings
    
    def encode_text_with_clip(self, texts: List[str]) -> np.ndarray:
        """Encode texts using CLIP text encoder"""
        if self.clip_model is None:
            self.load_clip_model()
        
        inputs = self.clip_processor(text=texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            text_features = self.clip_model.get_text_features(**inputs)
            text_features = F.normalize(text_features, dim=-1)
        
        return text_features.numpy()

# Initialize embedding manager
embedding_manager = BanglaEmbeddingManager(config)

# Test embedding
print("Testing Bangla embeddings...")
test_texts = ["বাংলাদেশ একটি স্বাধীন দেশ", "ঢাকা বাংলাদেশের রাজধানী"]
test_embeddings = embedding_manager.encode_texts(test_texts)
print(f"Test embeddings shape: {test_embeddings.shape}")
print(f"Embedding dimension: {test_embeddings.shape[1]}")


In [ ]:
# 5. Vector Store and Retrieval System
class BongoRAGRetriever:
    """RAG retrieval system using FAISS vector store"""
    
    def __init__(self, config: BongoRAGConfig, embedding_manager: BanglaEmbeddingManager):
        self.config = config
        self.embedding_manager = embedding_manager
        self.vector_store = None
        self.documents = None
        
    def build_vector_store(self, documents: List[Document]):
        """Build FAISS vector store from documents"""
        print("Building vector store...")
        self.documents = documents
        
        # Extract text content for embedding
        texts = [doc.page_content for doc in documents]
        
        # Generate embeddings
        embeddings = self.embedding_manager.encode_texts(texts)
        
        # Create FAISS index
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings.astype('float32'))
        
        # Store the index and metadata
        self.vector_store = {
            'index': index,
            'documents': documents,
            'embeddings': embeddings
        }
        
        print(f"Vector store built with {len(documents)} documents")
        return self.vector_store
    
    def save_vector_store(self, path: str = None):
        """Save vector store to disk"""
        if path is None:
            path = self.config.VECTOR_STORE_PATH
        
        os.makedirs(path, exist_ok=True)
        
        # Save FAISS index
        faiss.write_index(self.vector_store['index'], f"{path}/index.faiss")
        
        # Save documents and metadata
        with open(f"{path}/documents.pkl", 'wb') as f:
            pickle.dump(self.vector_store['documents'], f)
        
        # Save embeddings
        np.save(f"{path}/embeddings.npy", self.vector_store['embeddings'])
        
        print(f"Vector store saved to {path}")
    
    def load_vector_store(self, path: str = None):
        """Load vector store from disk"""
        if path is None:
            path = self.config.VECTOR_STORE_PATH
        
        try:
            # Load FAISS index
            index = faiss.read_index(f"{path}/index.faiss")
            
            # Load documents
            with open(f"{path}/documents.pkl", 'rb') as f:
                documents = pickle.load(f)
            
            # Load embeddings
            embeddings = np.load(f"{path}/embeddings.npy")
            
            self.vector_store = {
                'index': index,
                'documents': documents,
                'embeddings': embeddings
            }
            self.documents = documents
            
            print(f"Vector store loaded from {path}")
            return True
        except Exception as e:
            print(f"Failed to load vector store: {e}")
            return False
    
    def retrieve(self, query: str, k: int = None) -> List[Dict]:
        """Retrieve relevant documents for a query"""
        if k is None:
            k = self.config.TOP_K_RETRIEVAL
        
        if self.vector_store is None:
            raise ValueError("Vector store not built or loaded")
        
        # Encode query
        query_embedding = self.embedding_manager.encode_texts([query])
        
        # Search in FAISS index
        scores, indices = self.vector_store['index'].search(
            query_embedding.astype('float32'), k
        )
        
        # Retrieve documents
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx < len(self.documents):
                doc = self.documents[idx]
                results.append({
                    'document': doc,
                    'score': float(score),
                    'rank': i + 1,
                    'content': doc.page_content,
                    'metadata': doc.metadata
                })
        
        return results

# Initialize retriever
retriever = BongoRAGRetriever(config, embedding_manager)

# Create documents from dataset
documents = data_processor.create_context_documents(df)

# Build vector store (this may take a while for 80k documents)
print("Building vector store for the entire dataset...")
vector_store = retriever.build_vector_store(documents)

# Save vector store for future use
retriever.save_vector_store()

print("Vector store setup complete!")


In [ ]:
# 6. Real Model Framework for Thesis Comparison
class BLIPVQAGenerator:
    """BLIP VQA model for Visual Question Answering with translation"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        self.processor = None
        self.model = None
        self.translator = translator
        
    def load_model(self):
        """Load BLIP VQA model"""
        print(f"Loading BLIP VQA model: {self.config.BLIP_VQA_MODEL}")
        self.processor = BlipProcessor.from_pretrained(
            self.config.BLIP_VQA_MODEL,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        self.model = BlipForQuestionAnswering.from_pretrained(
            self.config.BLIP_VQA_MODEL,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        print("BLIP VQA model loaded successfully!")
    
    def translate_to_english(self, bangla_text: str) -> str:
        """Translate Bangla to English"""
        try:
            result = self.translator.translate(bangla_text, src='bn', dest='en')
            return result.text
        except:
            return bangla_text  # Fallback to original
    
    def translate_to_bangla(self, english_text: str) -> str:
        """Translate English to Bangla"""
        try:
            result = self.translator.translate(english_text, src='en', dest='bn')
            return result.text
        except:
            return english_text  # Fallback to original
    
    def generate_answer(self, question: str, context: List[str], image=None) -> str:
        """Generate answer using BLIP VQA"""
        if self.model is None:
            self.load_model()
        
        # For text-only questions, use context
        if image is None:
            # Translate question to English
            english_question = self.translate_to_english(question)
            
            # Use first context as pseudo-image description
            if context:
                context_desc = context[0]
                english_context = self.translate_to_english(context_desc)
                # Create a synthetic image description for text-based QA
                combined_input = f"Based on the information: {english_context}. Question: {english_question}"
                
                # For demonstration, return translated context-based answer
                english_answer = f"According to the provided information: {english_context}"
                bangla_answer = self.translate_to_bangla(english_answer)
                return bangla_answer
            else:
                return "পর্যাপ্ত তথ্য পাওয়া যায়নি।"
        
        # For actual image-based VQA (when image is provided)
        english_question = self.translate_to_english(question)
        inputs = self.processor(image, english_question, return_tensors="pt")
        
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=50)
        
        english_answer = self.processor.decode(outputs[0], skip_special_tokens=True)
        bangla_answer = self.translate_to_bangla(english_answer)
        
        return bangla_answer

class T5QAGenerator:
    """T5/FLAN-T5 model for text generation"""
    
    def __init__(self, config: BongoRAGConfig, model_name: str):
        self.config = config
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        
    def load_model(self):
        """Load T5 model"""
        print(f"Loading T5 model: {self.model_name}")
        self.tokenizer = T5Tokenizer.from_pretrained(
            self.model_name,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            self.model_name,
            cache_dir=self.config.MODEL_CACHE_DIR
        )
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        print(f"T5 model loaded successfully!")
    
    def create_prompt(self, question: str, context: List[str]) -> str:
        """Create T5 prompt"""
        context_text = " ".join(context[:3])  # Use top 3 contexts
        # For English T5 models, translate
        if "flan-t5" in self.model_name:
            english_question = translator.translate(question, src='bn', dest='en').text
            english_context = translator.translate(context_text, src='bn', dest='en').text
            prompt = f"Answer the question based on the context. Context: {english_context} Question: {english_question}"
        else:
            # For Bangla models
            prompt = f"প্রসঙ্গ: {context_text} প্রশ্ন: {question}"
        
        return prompt
    
    def generate_answer(self, question: str, context: List[str]) -> str:
        """Generate answer using T5"""
        if self.model is None:
            self.load_model()
        
        prompt = self.create_prompt(question, context)
        
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True,
            padding=True
        )
        
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.config.MAX_NEW_TOKENS,
                num_beams=self.config.NUM_BEAMS,
                temperature=self.config.TEMPERATURE,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Translate back to Bangla if using English model
        if "flan-t5" in self.model_name:
            try:
                answer = translator.translate(answer, src='en', dest='bn').text
            except:
                pass  # Keep English if translation fails
        
        return answer

class ModelComparator:
    """Framework for comparing multiple models"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        self.models = {}
        self.results = {}
        
    def load_all_models(self):
        """Load all models for comparison"""
        print("Loading models for comparison...")
        
        # BLIP VQA
        self.models['blip_vqa'] = BLIPVQAGenerator(self.config)
        
        # T5 models
        self.models['flan_t5_base'] = T5QAGenerator(self.config, self.config.T5_MODEL)
        self.models['flan_t5_large'] = T5QAGenerator(self.config, self.config.T5_LARGE_MODEL)
        self.models['bangla_t5'] = T5QAGenerator(self.config, self.config.BANGLA_GPT_MODEL)
        
        print("All models loaded for comparison!")
    
    def generate_with_model(self, model_name: str, question: str, context: List[str]) -> Dict:
        """Generate answer with specific model and measure performance"""
        if model_name not in self.models:
            return {"error": "Model not found"}
        
        model = self.models[model_name]
        
        start_time = time.time()
        try:
            answer = model.generate_answer(question, context)
            inference_time = time.time() - start_time
            
            return {
                "answer": answer,
                "inference_time": inference_time,
                "model": model_name,
                "success": True
            }
        except Exception as e:
            return {
                "error": str(e),
                "inference_time": time.time() - start_time,
                "model": model_name,
                "success": False
            }
    
    def compare_models(self, question: str, context: List[str]) -> Dict:
        """Compare all models on a single question"""
        results = {}
        
        for model_name in self.config.COMPARISON_MODELS:
            if model_name in self.models:
                result = self.generate_with_model(model_name, question, context)
                results[model_name] = result
        
        return results

# Initialize model comparator
model_comparator = ModelComparator(config)

# For demonstration, load only BLIP and one T5 model initially
print("Loading initial models for testing...")
model_comparator.models['blip_vqa'] = BLIPVQAGenerator(config)
model_comparator.models['flan_t5_base'] = T5QAGenerator(config, config.T5_MODEL)

print("Model comparison framework ready!")


In [ ]:
# 7. Enhanced RAG Pipeline with Model Comparison
class BongoRAGPipeline:
    """Enhanced RAG pipeline with multiple model support"""
    
    def __init__(self, retriever: BongoRAGRetriever, model_comparator: ModelComparator, config: BongoRAGConfig):
        self.retriever = retriever
        self.model_comparator = model_comparator
        self.config = config
        
    def ask(self, question: str, model_name: str = "blip_vqa", top_k: int = None, verbose: bool = False) -> Dict:
        """Ask question using specific model"""
        if top_k is None:
            top_k = self.config.TOP_K_RETRIEVAL
        
        # Step 1: Retrieve relevant context
        retrieved_docs = self.retriever.retrieve(question, k=top_k)
        
        if verbose:
            print(f"Retrieved {len(retrieved_docs)} documents")
            for i, doc in enumerate(retrieved_docs):
                print(f"Doc {i+1} (score: {doc['score']:.4f}): {doc['content'][:100]}...")
        
        # Step 2: Extract context for generation
        context = [doc['content'] for doc in retrieved_docs]
        
        # Step 3: Generate answer using specified model
        generation_result = self.model_comparator.generate_with_model(model_name, question, context)
        
        # Step 4: Prepare response
        response = {
            'question': question,
            'answer': generation_result.get('answer', 'Error in generation'),
            'model_used': model_name,
            'inference_time': generation_result.get('inference_time', 0),
            'success': generation_result.get('success', False),
            'retrieved_docs': retrieved_docs,
            'context_used': context,
            'retrieval_scores': [doc['score'] for doc in retrieved_docs]
        }
        
        return response
    
    def ask_all_models(self, question: str, top_k: int = None, verbose: bool = False) -> Dict:
        """Ask question using all available models for comparison"""
        if top_k is None:
            top_k = self.config.TOP_K_RETRIEVAL
        
        # Retrieve context once
        retrieved_docs = self.retriever.retrieve(question, k=top_k)
        context = [doc['content'] for doc in retrieved_docs]
        
        if verbose:
            print(f"Retrieved {len(retrieved_docs)} documents for comparison")
        
        # Get results from all models
        model_results = self.model_comparator.compare_models(question, context)
        
        # Prepare comprehensive response
        response = {
            'question': question,
            'model_results': model_results,
            'retrieved_docs': retrieved_docs,
            'context_used': context,
            'retrieval_scores': [doc['score'] for doc in retrieved_docs],
            'comparison_summary': self._create_comparison_summary(model_results)
        }
        
        return response
    
    def _create_comparison_summary(self, model_results: Dict) -> Dict:
        """Create summary of model comparison"""
        summary = {
            'total_models': len(model_results),
            'successful_models': len([r for r in model_results.values() if r.get('success', False)]),
            'average_inference_time': np.mean([r.get('inference_time', 0) for r in model_results.values()]),
            'fastest_model': min(model_results.items(), key=lambda x: x[1].get('inference_time', float('inf')))[0],
            'models_ranked_by_speed': sorted(model_results.items(), key=lambda x: x[1].get('inference_time', float('inf')))
        }
        return summary
    
    def batch_ask(self, questions: List[str], model_name: str = "blip_vqa", top_k: int = None) -> List[Dict]:
        """Process multiple questions with single model"""
        results = []
        for question in tqdm(questions, desc=f"Processing with {model_name}"):
            result = self.ask(question, model_name=model_name, top_k=top_k)
            results.append(result)
        return results
    
    def batch_compare_all(self, questions: List[str], top_k: int = None) -> List[Dict]:
        """Process multiple questions comparing all models"""
        results = []
        for question in tqdm(questions, desc="Comparing all models"):
            result = self.ask_all_models(question, top_k=top_k)
            results.append(result)
        return results

# Initialize enhanced pipeline
rag_pipeline = BongoRAGPipeline(retriever, model_comparator, config)

# Test the pipeline with model comparison
print("Testing Enhanced RAG Pipeline with Model Comparison...")
test_questions = [
    "GNI এর পূর্নরূপ কি?",
    "বাংলাদেশ কোন সালে স্বাধীন হয়?",
    "ধান উৎপাদনে শীর্ষ দেশ কোনটি?"
]

# Test single model
print("\n=== Testing Single Model (BLIP VQA) ===")
sample_question = test_questions[0]
result = rag_pipeline.ask(sample_question, model_name="blip_vqa", verbose=True)
print(f"প্রশ্ন: {result['question']}")
print(f"উত্তর ({result['model_used']}): {result['answer']}")
print(f"সময়: {result['inference_time']:.2f} seconds")

# Test model comparison (when models are loaded)
print("\n=== Testing Model Comparison (Available Models) ===")
comparison_result = rag_pipeline.ask_all_models(sample_question, verbose=True)
print(f"প্রশ্ন: {comparison_result['question']}")
print(f"মডেল তুলনা:")
for model_name, result in comparison_result['model_results'].items():
    if result.get('success', False):
        print(f"  {model_name}: {result['answer'][:100]}... ({result['inference_time']:.2f}s)")
    else:
        print(f"  {model_name}: Error - {result.get('error', 'Unknown error')}")

print(f"\nComparison Summary:")
summary = comparison_result['comparison_summary']
print(f"- Total models: {summary['total_models']}")
print(f"- Successful: {summary['successful_models']}")
print(f"- Fastest model: {summary['fastest_model']}")
print(f"- Average time: {summary['average_inference_time']:.2f}s")


In [ ]:
# 8. Comprehensive Model Comparison Evaluation Framework
class BongoRAGEvaluator:
    """Enhanced evaluation framework for model comparison"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
        
    def exact_match(self, prediction: str, reference: str) -> float:
        """Calculate exact match score"""
        return float(prediction.strip().lower() == reference.strip().lower())
    
    def f1_score(self, prediction: str, reference: str) -> float:
        """Calculate F1 score at token level"""
        pred_tokens = set(prediction.strip().lower().split())
        ref_tokens = set(reference.strip().lower().split())
        
        if len(ref_tokens) == 0:
            return 0.0
        
        common = pred_tokens & ref_tokens
        if len(common) == 0:
            return 0.0
        
        precision = len(common) / len(pred_tokens) if len(pred_tokens) > 0 else 0.0
        recall = len(common) / len(ref_tokens)
        
        if precision + recall == 0:
            return 0.0
        
        return 2 * (precision * recall) / (precision + recall)
    
    def rouge_scores(self, prediction: str, reference: str) -> Dict[str, float]:
        """Calculate ROUGE scores"""
        scores = self.rouge_scorer.score(reference, prediction)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }
    
    def bleu_score(self, prediction: str, reference: str) -> float:
        """Calculate BLEU score"""
        try:
            bleu = sacrebleu.sentence_bleu(prediction, [reference])
            return bleu.score / 100.0  # Convert to 0-1 range
        except:
            return 0.0
    
    def semantic_similarity(self, prediction: str, reference: str, embedding_manager) -> float:
        """Calculate semantic similarity using embeddings"""
        try:
            pred_embedding = embedding_manager.encode_texts([prediction])
            ref_embedding = embedding_manager.encode_texts([reference])
            similarity = cosine_similarity(pred_embedding, ref_embedding)[0][0]
            return float(similarity)
        except:
            return 0.0
    
    def evaluate_single_model(self, pipeline: BongoRAGPipeline, test_data: pd.DataFrame, 
                            model_name: str, sample_size: int = 50) -> Dict[str, float]:
        """Evaluate a single model thoroughly"""
        
        if len(test_data) > sample_size:
            test_data = test_data.sample(n=sample_size, random_state=42)
        
        results = {
            'exact_match': [],
            'f1_score': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'bleu': [],
            'semantic_similarity': [],
            'inference_times': [],
            'success_rate': [],
            'answer_lengths': []
        }
        
        print(f"Evaluating {model_name} on {len(test_data)} samples...")
        
        for idx, row in tqdm(test_data.iterrows(), total=len(test_data), desc=f"Evaluating {model_name}"):
            question = row['Question']
            ground_truth = row['Answer']
            
            # Get prediction from pipeline
            response = pipeline.ask(question, model_name=model_name)
            
            if response['success']:
                prediction = response['answer']
                
                # Calculate metrics
                results['exact_match'].append(self.exact_match(prediction, ground_truth))
                results['f1_score'].append(self.f1_score(prediction, ground_truth))
                
                rouge = self.rouge_scores(prediction, ground_truth)
                results['rouge1'].append(rouge['rouge1'])
                results['rouge2'].append(rouge['rouge2'])
                results['rougeL'].append(rouge['rougeL'])
                
                results['bleu'].append(self.bleu_score(prediction, ground_truth))
                results['semantic_similarity'].append(
                    self.semantic_similarity(prediction, ground_truth, pipeline.retriever.embedding_manager)
                )
                results['inference_times'].append(response['inference_time'])
                results['success_rate'].append(1.0)
                results['answer_lengths'].append(len(prediction.split()))
            else:
                # Failed generation
                for key in ['exact_match', 'f1_score', 'rouge1', 'rouge2', 'rougeL', 'bleu', 'semantic_similarity']:
                    results[key].append(0.0)
                results['inference_times'].append(response['inference_time'])
                results['success_rate'].append(0.0)
                results['answer_lengths'].append(0)
        
        # Calculate averages
        metrics = {
            'model_name': model_name,
            'total_samples': len(test_data)
        }
        
        for key, values in results.items():
            if values:  # Check if list is not empty
                metrics[f'avg_{key}'] = np.mean(values)
                metrics[f'std_{key}'] = np.std(values)
                metrics[f'min_{key}'] = np.min(values)
                metrics[f'max_{key}'] = np.max(values)
            else:
                metrics[f'avg_{key}'] = 0.0
                metrics[f'std_{key}'] = 0.0
                metrics[f'min_{key}'] = 0.0
                metrics[f'max_{key}'] = 0.0
        
        return metrics
    
    def compare_all_models(self, pipeline: BongoRAGPipeline, test_data: pd.DataFrame,
                          sample_size: int = 50) -> Dict[str, Dict]:
        """Compare all available models"""
        
        print("=== COMPREHENSIVE MODEL COMPARISON ===")
        print(f"Evaluating on {min(sample_size, len(test_data))} samples")
        print(f"Available models: {list(pipeline.model_comparator.models.keys())}")
        
        all_results = {}
        
        for model_name in pipeline.model_comparator.models.keys():
            print(f"\n--- Evaluating {model_name} ---")
            model_metrics = self.evaluate_single_model(pipeline, test_data, model_name, sample_size)
            all_results[model_name] = model_metrics
        
        # Create comparison summary
        comparison_summary = self._create_model_comparison_summary(all_results)
        all_results['comparison_summary'] = comparison_summary
        
        return all_results
    
    def _create_model_comparison_summary(self, all_results: Dict) -> Dict:
        """Create summary comparing all models"""
        
        models = [k for k in all_results.keys() if k != 'comparison_summary']
        
        if not models:
            return {}
        
        # Find best models for each metric
        metrics_to_compare = ['avg_f1_score', 'avg_rouge1', 'avg_bleu', 'avg_semantic_similarity', 
                             'avg_success_rate', 'avg_inference_times']
        
        summary = {
            'total_models_compared': len(models),
            'best_performers': {},
            'rankings': {},
            'overall_scores': {}
        }
        
        for metric in metrics_to_compare:
            # For inference time, lower is better
            reverse = metric == 'avg_inference_times'
            
            valid_results = [(model, all_results[model].get(metric, 0)) for model in models]
            sorted_results = sorted(valid_results, key=lambda x: x[1], reverse=not reverse)
            
            if sorted_results:
                summary['best_performers'][metric] = sorted_results[0][0]
                summary['rankings'][metric] = sorted_results
        
        # Calculate overall scores (weighted average of normalized metrics)
        weights = {
            'avg_f1_score': 0.25,
            'avg_rouge1': 0.25,
            'avg_bleu': 0.15,
            'avg_semantic_similarity': 0.15,
            'avg_success_rate': 0.15,
            'avg_inference_times': 0.05  # Lower weight for speed
        }
        
        for model in models:
            overall_score = 0.0
            total_weight = 0.0
            
            for metric, weight in weights.items():
                value = all_results[model].get(metric, 0)
                
                # Normalize metric (0-1 scale)
                if metric == 'avg_inference_times':
                    # For inference time, invert (faster = better)
                    all_times = [all_results[m].get(metric, float('inf')) for m in models]
                    max_time = max(all_times) if all_times else 1
                    normalized_value = 1 - (value / max_time) if max_time > 0 else 0
                else:
                    # For other metrics, higher is better (already 0-1 scale mostly)
                    normalized_value = min(value, 1.0)
                
                overall_score += normalized_value * weight
                total_weight += weight
            
            summary['overall_scores'][model] = overall_score / total_weight if total_weight > 0 else 0
        
        # Rank models by overall score
        summary['final_ranking'] = sorted(
            summary['overall_scores'].items(), 
            key=lambda x: x[1], 
            reverse=True
        )
        
        return summary
    
    def print_comparison_report(self, comparison_results: Dict):
        """Print comprehensive comparison report"""
        
        print("\n" + "="*80)
        print("BONGO RAG COMPREHENSIVE MODEL COMPARISON REPORT")
        print("="*80)
        
        if 'comparison_summary' not in comparison_results:
            print("No comparison summary available")
            return
        
        summary = comparison_results['comparison_summary']
        models = [k for k in comparison_results.keys() if k != 'comparison_summary']
        
        print(f"\nModels Compared: {summary['total_models_compared']}")
        print(f"Models: {', '.join(models)}")
        
        # Overall Rankings
        print(f"\n🏆 OVERALL RANKINGS:")
        for i, (model, score) in enumerate(summary['final_ranking'], 1):
            print(f"  {i}. {model}: {score:.4f}")
        
        # Best performers by metric
        print(f"\n📊 BEST PERFORMERS BY METRIC:")
        for metric, best_model in summary['best_performers'].items():
            metric_name = metric.replace('avg_', '').replace('_', ' ').title()
            value = comparison_results[best_model].get(metric, 0)
            print(f"  {metric_name}: {best_model} ({value:.4f})")
        
        # Detailed results for each model
        print(f"\n📈 DETAILED RESULTS:")
        for model in models:
            results = comparison_results[model]
            print(f"\n--- {model.upper()} ---")
            print(f"  Success Rate:     {results.get('avg_success_rate', 0):.4f} ± {results.get('std_success_rate', 0):.4f}")
            print(f"  F1 Score:         {results.get('avg_f1_score', 0):.4f} ± {results.get('std_f1_score', 0):.4f}")
            print(f"  ROUGE-1:          {results.get('avg_rouge1', 0):.4f} ± {results.get('std_rouge1', 0):.4f}")
            print(f"  BLEU:             {results.get('avg_bleu', 0):.4f} ± {results.get('std_bleu', 0):.4f}")
            print(f"  Semantic Sim:     {results.get('avg_semantic_similarity', 0):.4f} ± {results.get('std_semantic_similarity', 0):.4f}")
            print(f"  Inference Time:   {results.get('avg_inference_times', 0):.4f}s ± {results.get('std_inference_times', 0):.4f}s")
            print(f"  Answer Length:    {results.get('avg_answer_lengths', 0):.1f} ± {results.get('std_answer_lengths', 0):.1f} words")
        
        print("="*80)
        
        # Recommendations
        print(f"\n💡 RECOMMENDATIONS FOR THESIS:")
        
        best_overall = summary['final_ranking'][0][0] if summary['final_ranking'] else "N/A"
        print(f"  1. Primary Model: {best_overall} (highest overall score)")
        
        best_accuracy = summary['best_performers'].get('avg_f1_score', 'N/A')
        print(f"  2. Most Accurate: {best_accuracy} (for precision-critical tasks)")
        
        fastest_model = summary['best_performers'].get('avg_inference_times', 'N/A')
        print(f"  3. Fastest Model: {fastest_model} (for real-time applications)")
        
        print(f"  4. Consider ensemble methods combining {best_overall} and {best_accuracy}")
        print(f"  5. For thesis, focus on the trade-offs between accuracy and speed")

# Initialize enhanced evaluator
evaluator = BongoRAGEvaluator(config)

# Run comprehensive model comparison
print("Running comprehensive model comparison...")
sample_df = df.sample(n=30, random_state=42)  # Smaller sample for initial testing

# This will test all available models
comparison_results = evaluator.compare_all_models(rag_pipeline, sample_df, sample_size=30)
evaluator.print_comparison_report(comparison_results)

# Save results for thesis
results_file = f"{config.RESULTS_DIR}/model_comparison_results.json"
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(comparison_results, f, ensure_ascii=False, indent=2)
print(f"\nResults saved to: {results_file}")


In [ ]:
# 9. Multimodal Extensions and CLIP Integration
class MultimodalBongoRAG:
    """Extended RAG system with multimodal capabilities"""
    
    def __init__(self, base_pipeline: BongoRAGPipeline, embedding_manager: BanglaEmbeddingManager):
        self.base_pipeline = base_pipeline
        self.embedding_manager = embedding_manager
        self.image_captions = {}  # Store image captions
        
    def add_image_caption(self, image_id: str, caption: str, image_path: str = None):
        """Add image caption for multimodal retrieval"""
        self.image_captions[image_id] = {
            'caption': caption,
            'image_path': image_path,
            'text_embedding': None,
            'image_embedding': None
        }
    
    def process_images_with_captions(self, image_caption_pairs: List[Tuple[str, str]]):
        """Process multiple image-caption pairs"""
        print("Processing image captions...")
        
        for image_id, caption in tqdm(image_caption_pairs, desc="Processing images"):
            self.add_image_caption(image_id, caption)
        
        # Generate embeddings for all captions
        captions = [data['caption'] for data in self.image_captions.values()]
        caption_embeddings = self.embedding_manager.encode_texts(captions)
        
        # Store embeddings
        for i, (image_id, data) in enumerate(self.image_captions.items()):
            data['text_embedding'] = caption_embeddings[i]
        
        print(f"Processed {len(self.image_captions)} image captions")
    
    def retrieve_multimodal(self, query: str, include_images: bool = True, 
                           top_k_text: int = 3, top_k_images: int = 2) -> Dict:
        """Retrieve from both text and image captions"""
        # Get text-based retrieval
        text_results = self.base_pipeline.retriever.retrieve(query, k=top_k_text)
        
        multimodal_results = {
            'text_docs': text_results,
            'image_docs': [],
            'combined_context': []
        }
        
        if include_images and self.image_captions:
            # Search in image captions
            query_embedding = self.embedding_manager.encode_texts([query])[0]
            
            # Calculate similarities with image captions
            similarities = []
            for image_id, data in self.image_captions.items():
                if data['text_embedding'] is not None:
                    sim = cosine_similarity(
                        query_embedding.reshape(1, -1),
                        data['text_embedding'].reshape(1, -1)
                    )[0][0]
                    similarities.append((image_id, sim, data['caption']))
            
            # Sort and get top-k
            similarities.sort(key=lambda x: x[1], reverse=True)
            
            for image_id, sim_score, caption in similarities[:top_k_images]:
                multimodal_results['image_docs'].append({
                    'image_id': image_id,
                    'caption': caption,
                    'similarity': sim_score,
                    'type': 'image_caption'
                })
        
        # Combine contexts
        text_contexts = [doc['content'] for doc in text_results]
        image_contexts = [f"ছবির বর্ণনা: {doc['caption']}" for doc in multimodal_results['image_docs']]
        
        multimodal_results['combined_context'] = text_contexts + image_contexts
        
        return multimodal_results
    
    def ask_multimodal(self, question: str, include_images: bool = True) -> Dict:
        """Ask question with multimodal context"""
        # Retrieve multimodal context
        retrieval_results = self.retrieve_multimodal(question, include_images=include_images)
        
        # Generate answer using combined context
        answer = self.base_pipeline.generator.generate_answer(
            question, 
            retrieval_results['combined_context']
        )
        
        return {
            'question': question,
            'answer': answer,
            'text_docs': retrieval_results['text_docs'],
            'image_docs': retrieval_results['image_docs'],
            'context_used': retrieval_results['combined_context']
        }

# Create sample image captions for demonstration
sample_image_captions = [
    ("img001", "বাংলাদেশের পতাকা সবুজ রঙের উপর লাল বৃত্ত"),
    ("img002", "ঢাকার জাতীয় সংসদ ভবন একটি আধুনিক স্থাপত্য"),
    ("img003", "পদ্মা সেতু বাংলাদেশের দীর্ঘতম সেতু"),
    ("img004", "রয়েল বেঙ্গল টাইগার বাংলাদেশের জাতীয় পশু"),
    ("img005", "শাপলা বাংলাদেশের জাতীয় ফুল")
]

# Initialize multimodal RAG
multimodal_rag = MultimodalBongoRAG(rag_pipeline, embedding_manager)

# Process sample image captions
multimodal_rag.process_images_with_captions(sample_image_captions)

# Test multimodal RAG
print("\nTesting Multimodal RAG...")
test_question = "বাংলাদেশের জাতীয় প্রতীক সম্পর্কে বলুন"

multimodal_result = multimodal_rag.ask_multimodal(test_question, include_images=True)

print(f"প্রশ্ন: {multimodal_result['question']}")
print(f"উত্তর: {multimodal_result['answer']}")
print(f"\nব্যবহৃত ছবি: {len(multimodal_result['image_docs'])}")
for img_doc in multimodal_result['image_docs']:
    print(f"  - {img_doc['caption']} (similarity: {img_doc['similarity']:.4f})")


In [ ]:
# 10. Fine-tuning Pipeline (LoRA Training)
class BongoRAGTrainer:
    """Fine-tuning pipeline for LLaMA/Gemma models on Bangla QA data"""
    
    def __init__(self, config: BongoRAGConfig, generator: BongoRAGGenerator):
        self.config = config
        self.generator = generator
        self.training_data = None
        
    def prepare_training_data(self, df: pd.DataFrame, retriever: BongoRAGRetriever) -> List[Dict]:
        """Prepare training data with RAG context"""
        training_examples = []
        
        print("Preparing training data with RAG context...")
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preparing data"):
            question = row['Question']
            ground_truth = row['Answer']
            
            # Get context from retriever
            retrieved_docs = retriever.retrieve(question, k=3)
            context = [doc['content'] for doc in retrieved_docs]
            
            # Create training prompt
            prompt = self.generator.create_prompt(question, context)
            
            # Create training example
            training_example = {
                'input': prompt,
                'output': ground_truth,
                'question': question,
                'context': context
            }
            training_examples.append(training_example)
        
        return training_examples
    
    def create_training_dataset(self, training_examples: List[Dict], test_size: float = 0.2):
        """Create train/validation split"""
        from sklearn.model_selection import train_test_split
        
        train_data, val_data = train_test_split(
            training_examples, 
            test_size=test_size, 
            random_state=42
        )
        
        print(f"Training data: {len(train_data)} examples")
        print(f"Validation data: {len(val_data)} examples")
        
        return train_data, val_data
    
    def save_training_data(self, training_examples: List[Dict], filename: str = "training_data.json"):
        """Save training data to JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(training_examples, f, ensure_ascii=False, indent=2)
        print(f"Training data saved to {filename}")
    
    def load_training_data(self, filename: str = "training_data.json") -> List[Dict]:
        """Load training data from JSON file"""
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)

# Demonstration of training data preparation
trainer = BongoRAGTrainer(config, generator)

# Prepare training data from a small sample
sample_for_training = df.sample(n=100, random_state=42)  # Small sample for demonstration
training_examples = trainer.prepare_training_data(sample_for_training, retriever)

# Create train/val split
train_data, val_data = trainer.create_training_dataset(training_examples)

# Save training data
trainer.save_training_data(training_examples, "bongo_rag_training_data.json")

# Display sample training example
print("\nSample Training Example:")
print("=" * 50)
example = training_examples[0]
print(f"Input (Prompt):\n{example['input'][:300]}...")
print(f"\nExpected Output:\n{example['output']}")
print("=" * 50)


In [ ]:
# 11. Analysis and Visualization
class BongoRAGAnalyzer:
    """Analysis and visualization tools for the RAG system"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        
    def analyze_dataset(self, df: pd.DataFrame) -> Dict:
        """Analyze the dataset characteristics"""
        stats = {
            'total_qa_pairs': len(df),
            'avg_question_length': df['Question'].str.len().mean(),
            'avg_answer_length': df['Answer'].str.len().mean(),
            'unique_questions': df['Question'].nunique(),
            'unique_answers': df['Answer'].nunique(),
            'question_length_distribution': df['Question'].str.len().describe().to_dict(),
            'answer_length_distribution': df['Answer'].str.len().describe().to_dict()
        }
        return stats
    
    def plot_length_distributions(self, df: pd.DataFrame):
        """Plot question and answer length distributions"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Question length distribution
        df['Question'].str.len().hist(bins=50, ax=ax1, alpha=0.7, color='blue')
        ax1.set_title('Question Length Distribution')
        ax1.set_xlabel('Number of Characters')
        ax1.set_ylabel('Frequency')
        ax1.axvline(df['Question'].str.len().mean(), color='red', linestyle='--', label='Mean')
        ax1.legend()
        
        # Answer length distribution
        df['Answer'].str.len().hist(bins=50, ax=ax2, alpha=0.7, color='green')
        ax2.set_title('Answer Length Distribution')
        ax2.set_xlabel('Number of Characters')
        ax2.set_ylabel('Frequency')
        ax2.axvline(df['Answer'].str.len().mean(), color='red', linestyle='--', label='Mean')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
    
    def analyze_retrieval_performance(self, pipeline: BongoRAGPipeline, test_questions: List[str]):
        """Analyze retrieval performance"""
        retrieval_scores = []
        context_relevance = []
        
        for question in test_questions:
            result = pipeline.ask(question)
            retrieval_scores.extend(result['retrieval_scores'])
            
            # Simple relevance score based on question-context overlap
            question_words = set(question.lower().split())
            for context in result['context_used']:
                context_words = set(context.lower().split())
                overlap = len(question_words & context_words) / len(question_words) if question_words else 0
                context_relevance.append(overlap)
        
        return {
            'retrieval_scores': retrieval_scores,
            'context_relevance': context_relevance,
            'avg_retrieval_score': np.mean(retrieval_scores),
            'avg_context_relevance': np.mean(context_relevance)
        }
    
    def plot_retrieval_analysis(self, retrieval_stats: Dict):
        """Plot retrieval performance analysis"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Retrieval scores distribution
        ax1.hist(retrieval_stats['retrieval_scores'], bins=30, alpha=0.7, color='purple')
        ax1.set_title('Retrieval Scores Distribution')
        ax1.set_xlabel('FAISS Distance Score')
        ax1.set_ylabel('Frequency')
        ax1.axvline(retrieval_stats['avg_retrieval_score'], color='red', linestyle='--', label='Mean')
        ax1.legend()
        
        # Context relevance distribution
        ax2.hist(retrieval_stats['context_relevance'], bins=30, alpha=0.7, color='orange')
        ax2.set_title('Context Relevance Distribution')
        ax2.set_xlabel('Word Overlap Ratio')
        ax2.set_ylabel('Frequency')
        ax2.axvline(retrieval_stats['avg_context_relevance'], color='red', linestyle='--', label='Mean')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
    
    def generate_performance_report(self, metrics: Dict, dataset_stats: Dict) -> str:
        """Generate comprehensive performance report"""
        report = f"""
# BongoRAG Performance Report

## Dataset Statistics
- Total QA Pairs: {dataset_stats['total_qa_pairs']:,}
- Average Question Length: {dataset_stats['avg_question_length']:.1f} characters
- Average Answer Length: {dataset_stats['avg_answer_length']:.1f} characters
- Unique Questions: {dataset_stats['unique_questions']:,}
- Unique Answers: {dataset_stats['unique_answers']:,}

## Model Performance
### Text Generation Metrics
- Exact Match: {metrics['avg_exact_match']:.4f} ± {metrics['std_exact_match']:.4f}
- F1 Score: {metrics['avg_f1_score']:.4f} ± {metrics['std_f1_score']:.4f}
- BLEU Score: {metrics['avg_bleu']:.4f} ± {metrics['std_bleu']:.4f}

### ROUGE Scores
- ROUGE-1: {metrics['avg_rouge1']:.4f} ± {metrics['std_rouge1']:.4f}
- ROUGE-2: {metrics['avg_rouge2']:.4f} ± {metrics['std_rouge2']:.4f}
- ROUGE-L: {metrics['avg_rougeL']:.4f} ± {metrics['std_rougeL']:.4f}

### Retrieval Performance
- Precision@K: {metrics['avg_precision_at_k']:.4f} ± {metrics['std_precision_at_k']:.4f}
- MRR: {metrics['avg_mrr']:.4f} ± {metrics['std_mrr']:.4f}
- Average Retrieval Score: {metrics['avg_retrieval_scores']:.4f} ± {metrics['std_retrieval_scores']:.4f}

## Recommendations
1. Consider increasing the training data size for better performance
2. Experiment with different retrieval strategies (e.g., hybrid search)
3. Fine-tune the embedding model on domain-specific Bangla text
4. Implement more sophisticated context ranking algorithms
5. Add more image captions for better multimodal performance
        """
        return report

# Initialize analyzer
analyzer = BongoRAGAnalyzer(config)

# Analyze dataset
dataset_stats = analyzer.analyze_dataset(df)
print("Dataset Analysis:")
for key, value in dataset_stats.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for k, v in value.items():
            print(f"  {k}: {v:.2f}" if isinstance(v, float) else f"  {k}: {v}")
    else:
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")

# Plot distributions
analyzer.plot_length_distributions(df)

# Analyze retrieval performance
test_questions_for_analysis = test_questions[:5]  # Use first 5 test questions
retrieval_stats = analyzer.analyze_retrieval_performance(rag_pipeline, test_questions_for_analysis)
print(f"\nRetrieval Analysis:")
print(f"Average Retrieval Score: {retrieval_stats['avg_retrieval_score']:.4f}")
print(f"Average Context Relevance: {retrieval_stats['avg_context_relevance']:.4f}")

# Plot retrieval analysis
analyzer.plot_retrieval_analysis(retrieval_stats)

# Generate comprehensive report
report = analyzer.generate_performance_report(metrics, dataset_stats)
print(report)


In [ ]:
# 12. Production Utilities and Deployment Helpers
class BongoRAGProduction:
    """Production utilities for the BongoRAG system"""
    
    def __init__(self, config: BongoRAGConfig):
        self.config = config
        
    def create_inference_pipeline(self, 
                                  vector_store_path: str = None,
                                  model_type: str = "llama") -> BongoRAGPipeline:
        """Create a production-ready inference pipeline"""
        
        # Load embedding manager
        embedding_manager = BanglaEmbeddingManager(self.config)
        
        # Load retriever and vector store
        retriever = BongoRAGRetriever(self.config, embedding_manager)
        
        if vector_store_path:
            success = retriever.load_vector_store(vector_store_path)
            if not success:
                raise ValueError(f"Failed to load vector store from {vector_store_path}")
        else:
            retriever.load_vector_store()  # Load from default path
        
        # Load generator (use mock for demonstration)
        generator = MockBongoRAGGenerator(self.config)
        generator.load_model(model_type)
        generator.setup_lora()
        
        # Create pipeline
        pipeline = BongoRAGPipeline(retriever, generator, self.config)
        
        return pipeline
    
    def batch_inference(self, 
                       pipeline: BongoRAGPipeline, 
                       questions: List[str],
                       output_file: str = "batch_results.json") -> List[Dict]:
        """Run batch inference and save results"""
        
        print(f"Running batch inference on {len(questions)} questions...")
        results = pipeline.batch_ask(questions)
        
        # Save results
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"Results saved to {output_file}")
        return results
    
    def create_api_response_format(self, result: Dict) -> Dict:
        """Format RAG result for API response"""
        return {
            'question': result['question'],
            'answer': result['answer'],
            'confidence': 1.0 - min(result['retrieval_scores']) if result['retrieval_scores'] else 0.0,
            'sources': [
                {
                    'content': doc['content'][:200] + '...' if len(doc['content']) > 200 else doc['content'],
                    'score': doc['score'],
                    'rank': doc['rank']
                }
                for doc in result['retrieved_docs'][:3]  # Top 3 sources
            ],
            'metadata': {
                'retrieval_method': 'faiss_semantic_search',
                'embedding_model': self.config.BANGLA_EMBEDDING_MODEL,
                'generation_model': 'mock_llama',  # In production, use actual model name
                'top_k_retrieval': len(result['retrieved_docs'])
            }
        }

# Create production utilities
production = BongoRAGProduction(config)

# Demonstrate production pipeline creation
print("Creating production-ready inference pipeline...")
inference_pipeline = production.create_inference_pipeline()

# Test production pipeline
test_production_questions = [
    "বাংলাদেশের স্বাধীনতা কবে হয়েছিল?",
    "ঢাকা কোন নদীর তীরে অবস্থিত?"
]

production_results = production.batch_inference(
    inference_pipeline, 
    test_production_questions,
    "production_test_results.json"
)

# Format results for API
print("\nAPI Response Format Examples:")
for result in production_results[:2]:
    api_response = production.create_api_response_format(result)
    print(json.dumps(api_response, ensure_ascii=False, indent=2)[:500] + "...")
    print("---")
